In [276]:
from DrissionPage import ChromiumPage
from urllib.parse import unquote
import time
from lxml import  etree


In [277]:
page = ChromiumPage()
page.listen.start(['group/singleresult', 'kns8s/brief/grid'])
page.get('https://kns.cnki.net/kns8s/AdvSearch')
print('等待搜索')
while True:
    arti = page.eles('xpath://*[@id="gridTable"]/div/div/div/table/tbody/tr[1]', timeout=1)
    if arti:
        break
print('搜索完成')

等待搜索
搜索完成


获取各分类数量

In [278]:
category_count= {}
doctype_menus = page.eles('xpath://ul[contains(@class,"doctype-menu")]//a[@name="classify"]', timeout=2)
for i in doctype_menus:
    t = i.ele('xpath://span').text
    n = i.ele('xpath://em').text
    category_count[t] = n
    print(t,n)

学术期刊 119
学位论文 0
学位论文 0
博士 0
硕士 0
会议 41
会议 41
国内会议 40
国际会议 1
会议视频 0
报纸 0
年鉴 
图书 0
图书 0
中文图书 0
外文图书 0
专利 
专利 
中国专利 
境外专利 
标准 0
标准 0
国家标准 0
行业标准 0
标准题录 0
成果 0
成果 0
学术辑刊 0
特色期刊 0
视频 0
文库 
学术期刊 119
学位论文 0
学位论文 0
博士 0
硕士 0
会议 41
会议 41
国内会议 40
国际会议 1
会议视频 0
报纸 0
年鉴 
图书 0
图书 0
中文图书 0
外文图书 0
专利 
专利 
中国专利 
境外专利 
标准 0
标准 0
国家标准 0
行业标准 0
标准题录 0
成果 0
学术辑刊 0
特色期刊 0
视频 0
文库 


侧边计数点击

In [279]:
divGroups = page.eles('xpath://div[@id="divGroup"]/dl/dt[@class="tit"]/b')
for i in divGroups:
    print(i.text)
    i.click(by_js=True)

主题
来源类别
学科
研究层次
年度
文献类型
文献来源
作者
机构
基金
OA出版


翻页

In [280]:
while True:
    PageNext = page.ele('#PageNext', timeout=2)
    if PageNext:
        PageNext.click()
        time.sleep(1)
    else:
        break

获取监听数据

In [281]:
subject_count = {} # 左侧分类计数
articles = [] # 文章列表

for i in page.listen.steps(timeout=2):
    url = i.request.url
    data = i.response.body
    if 'kns8s/brief/grid' in url: # 主体内容
        h = etree.HTML(data)
        items = h.xpath('//table[@class="result-table-list"]/tbody/tr')
        for item in items:
            temp = {}
            temp['url'] = item.xpath('./td[@class="name"]//a/@href')[0]
            temp['title'] = item.xpath('./td[@class="name"]//a/text()')[0]
            
            
            authors = []
            author = item.xpath('./td[@class="author"]//a')
            for i in author:
                author_url = i.xpath('./@href')[0]
                author_name = i.xpath('.//text()')[0]
                if 'javascript' in author_url:
                    authors = [{'name': i, 'url': None}  for i in ''.join(i.xpath('.//text()')).split('，') ]
                else:
                    authors.append({'name': author_name, 'url': author_url})
                
            temp['author'] = item.xpath('./td[@class="author"]//a//text()')
            temp['source'] = item.xpath('./td[@class="source"]//a/text()')[0]
            temp['date'] = item.xpath('./td[@class="date"]//text()')[0] # 时间
            temp['data'] = item.xpath('./td[@class="data"]//span/text()') # 数据库
            temp['data'] = temp['data'][0] if temp['data'] else None
            temp['quote'] = item.xpath('./td[@class="quote"]//span/text()')
            temp['quote'] = temp['quote'][0] if temp['quote'] else None
            temp['download'] = item.xpath('./td[@class="date"]//a/text()') # 下载
            temp['download'] = temp['download'][0] if temp['download'] else None
            articles.append(temp) 
    elif 'group/singleresult' in url: # 左侧计数信息
        print(data)
        h = etree.HTML(data)
        tit = h.xpath('//dd/@tit')[0]
        items = h.xpath('//li')
        temp = {}
        for item in items:
            name = item.xpath('./a/text()')[0]
            value = item.xpath('./span/text()')[0]
            value = value.replace('(','').replace(')','')
            if '万' in value:
                value = float(value.replace('万',''))*10000
            temp[name] = value
        subject_count[tit] = temp
        

<dd field="YE" tit="年度"><div class="resultlist" onmouseout="mouseout(this)" onmouseenter="resultHide($(this))"><ul><li style="display:list-item"><input type="checkbox" value="2022" text="2022年" title="2022年"> <a href="javascript:void(0)" title="2022年">2022年</a> <span>(3)</span></li><li style="display:list-item"><input type="checkbox" value="2021" text="2021年" title="2021年"> <a href="javascript:void(0)" title="2021年">2021年</a> <span>(4)</span></li><li style="display:list-item"><input type="checkbox" value="2020" text="2020年" title="2020年"> <a href="javascript:void(0)" title="2020年">2020年</a> <span>(2)</span></li><li style="display:list-item"><input type="checkbox" value="2019" text="2019年" title="2019年"> <a href="javascript:void(0)" title="2019年">2019年</a> <span>(2)</span></li><li style="display:list-item"><input type="checkbox" value="2018" text="2018年" title="2018年"> <a href="javascript:void(0)" title="2018年">2018年</a> <span>(3)</span></li><li style="display:list-item"><input type="ch

计量可视化分析

In [ ]:
anisys_all = page.ele('xpath://li[@id="anisys_all"]')
anisys_all.click(by_js=True)
time.sleep(1)
anisys_tab = page.get_tab(title='计量可视化分析')
anisys_tab.listen.start(['getGroupData'])
time.sleep(1)
anisys_tab.refresh()
time.sleep(1)
anisys_tab.refresh()
time.sleep(1)
anisys_data = {}
for i in anisys_tab.listen.steps(timeout=2):
    url = i.request.url
    postData =unquote(i.request.postData) 
    data = i.response.body
    for i in data:
        if 'y' in i:
            i['value'] = i['y']
        if ''
            ...
    if 'getGroupData' in url:
        anisys_data[postData.split('&')[3].split('=')[1]] = data

获取用户信息

In [283]:
try:
    author_tab = page.get_tab(title='作者')
except:
    author_tab = page.new_tab()
author_tab.listen.start('experts')
author_tab.get('https://kns.cnki.net/kcms2/author/detail?v=Nyg97wmOeE7B23i7DB-Wq31WeINiF06xmgTmoBoMx0sXtkAxZJoDot5NMIzLQfaSh-HhvifSwPKRh-ZDhdGYQAYlnz35f_S2ZwyPVVXbED3805Z7yyKKIC4yiYR3YObe&uniplatform=NZKPT&language=CHS')
for i in range(50):
    time.sleep(1)
    author_tab.scroll.to_bottom()
    recvideotitle = author_tab.ele('xpath://h2[@id="recvideotitle"]', timeout=0.5)
    if recvideotitle:
        print('到底了')
        break

author_data = {}
author_data['names'] = {
    'detail': '作者详情',
    'query': '同名作者',
    'domains': '作者关注领域',
    'quote_rank': '最高被引',
    'download_rank': '最高下载',
    'journal': '发表在期刊的文献',
    'meeting': '发表在会议的文献',
    'thesis': '发表在硕博的文献',
    'newspaper': '发表在报纸的文献',
    'max_pubjournals': '最高发表期刊',
    'max_pubconferences': '最高发表会议',
    'max_pubdegreeunits': '最高发表硕博',
    'max_pubnewspapers': '最高发表报纸',
    'overReferences':'曾参考文献',
    'tutors':'导师',
    'coauthors':'合作者',
    'funds': '作者基金',
    'videos': '视频'
    
    
    
}
for i in author_tab.listen.steps(timeout=2):
    url = i.request.url
    t = url.split('?')[0].split('/')[-1]
    data = i.response.body
    params = i.request.params
    if 'detail' in url: # 作者详情
        author_data['detail'] = data.get('data',{})
    elif 'query' in url: # 同名作者
        author_data['query'] = data.get('data',{})
    elif 'domains' in url: #作者关注领域
        author_data['domains'] = data.get('data',{}).get('data',{})
    elif 'resources' in url: 
        
        resource = params.get('resource')
        sequence = params.get('sequence')
        if resource == 'SCDB' and sequence == 'CF': # 最高被引
            author_data['quote_rank'] = data.get('data',{}).get('data',{})
        elif resource == 'SCDB' and sequence == 'DFR': # 最高下载
            author_data['download_rank'] = data.get('data',{}).get('data',{})
        elif resource == 'CJFD' and sequence == 'PT': # 发表在期刊的文献
            author_data['journal'] = data.get('data',{}).get('data',{})
        elif resource == 'CIPD' and sequence == 'PT': # 发表在会议的文献
            author_data['meeting'] = data.get('data',{}).get('data',{})
        elif resource == 'CDMD' and sequence == 'PT': # 发表在硕博的文献
            author_data['thesis'] = data.get('data',{}).get('data',{})
        elif resource == 'CCND' and sequence == 'PT': # 发表在报纸的文献
            author_data['newspaper'] = data.get('data',{}).get('data',{})
    elif 'publications' in url: # 发表文献
        if params.get('type') == 'journals': # 最高发表期刊
            author_data['max_pubjournals'] = data.get('data',{}).get('data',{})
        elif params.get('type') == 'conferences': # 最高发表会议
            author_data['max_pubconferences'] = data.get('data',{}).get('data',{})
        elif params.get('type') == 'degreeunits': # 最高发表在硕博的文献的单位
            author_data['max_pubdegreeunits'] = data.get('data',{}).get('data',{})
        elif params.get('type') == 'newspapers': # 最高发表在报纸的文献的单位
            author_data['max_pubnewspapers'] = data.get('data',{}).get('data',{})
    elif 'literature/overReferences' in url: # 曾参考文献
        author_data['overReferences'] = data.get('data',{}).get('data',{})
    elif 'tutors' in url: # 作者导师
        author_data['tutors'] = data.get('data',[])
    elif 'coauthors' in url: # 作者合作
        author_data['coauthors'] = data.get('data',{}).get('data',{})
    elif 'funds' in url: # 作者基金
        author_data['funds'] = data.get('data',{}).get('data',{})
    elif 'videos' in url: # 作者视频
        author_data['videos'] = data.get('data',{}).get('data',{})


到底了


In [284]:
try:
    article_tab = page.get('article/abstract')
except:
    article_tab = page.new_tab()
article_tab.listen.start(['references'])
for i in articles:
    article_tab.get(i['url'])
    time.sleep(1)
    

In [305]:
try:
    article_tab = page.get(url='article/abstract')
except:
    article_tab = page.new_tab()
for index,a in enumerate(articles):
    if not a.get('detail'):
        time.sleep(3)
        article_tab.get(a['url'])
        print(a['title'], a['url'])
        desc = article_tab.eles('xpath://div[@class="doc"]//div[@class="row"]')
        r = []
        for i in (desc):
            # print(i.text)
            if '摘要' in i.text:
                r.append('摘要')
                r.append(i.text.split('摘要', 1)[1].strip())
            elif '：' in i.text or '\n' in i.text:
                for j in i.text.split('：'):
                    for k in j.split('\n'):
                        if k != '':
                            r.append (k.strip())
            else:
                r.append(i.text.strip())
        r = {r[i]:r[i+1] for i in range(0,len(r),2)}
        print(r)
        articles[index]['detail'] = r


基于条件随机场的文献记录分析算法 https://kns.cnki.net/kcms2/article/abstract?v=Nyg97wmOeE7IfQ11DkIOU0bLneaouRx2u2_LuTOBdfHuHSLOfD0PgbUPoQPbTUfg0U3OTELfWCSW_JY4kDbWAiH94K96dD29qxVUG1tSQwVDrcsBkJnnm__Pb5IjRMCERJOOvHS1RupHr6cVI--tKFdNfv6gMn-wtWk-g0OXotI_PmnqksC2TEpakupmyhFPydziu3bsEsM=&uniplatform=NZKPT&language=CHS
{'摘要': '：随着互联网的快速发展,网页已成为获取信息的主要来源。为了使出版机构能够及时从大量网页中发现所需文献,需要设计能够从HTML页面中自动提取文献信息的算法。为此,本文设计了基于条件随机场的文献记录分析算法:首先,设计了文档对象树的分割算法,通过分割标记将页面数据分成独立的部分,这些数据块由标签和文本序列构成;随后,将该序列作为条件随机场模型的特征向量,建立文献信息标记模型;最后,设计启发式算法,从标记模型中提取文献信息数据,并通过实验验证了其有效性。更多还原', '关键词': '数字出版;条件随机场;文献记录分析;', '基金': 'r1601000668,北京印刷学院博士启动基金项目,ea201507,北京印刷学院校级重点项目；PXM2016_014223_000025,协同创新中心一绿色印刷与出版技术；', '会议名称': '2016年全国通信软件学术会议', '会议时间': '2016-06-24', '会议地点': '中国陕西西安', '专辑': '信息科技', '专题': '计算机软件及计算机应用', '分类号': 'TP391.1'}
基于维基百科层次分类框架的主题推荐系统的研究 https://kns.cnki.net/kcms2/article/abstract?v=Nyg97wmOeE4UkQFXzmJZGV_hBMMVmlN43lAI9DY47d0fcVKDm8hzePYLISvj4hghFlEMXisHrEYumYRfhUPjXC09bopWF2AOg4nlaYADpxhCnX6VbebY5U2qMgucC-ve20KxVaYcnySlRV4R

KeyboardInterrupt: 

In [296]:
articles

[{'url': 'https://kns.cnki.net/kcms2/article/abstract?v=Nyg97wmOeE707ONVGBvquN7xbyQ7mjOOQutKOEdNBAqRqw0nwqSza1nsyE5hgmqOi7TMSlG1vBe_Y1sK4JTmzno__tSpphbWt81cl8Z48h5IHgZpWMciY3sxrEcHNArH-zXXZ7OL2qnGCh-yVCIngh2B8I3xbNNhgnO_545UNlBJBKiKPtNWGmfmbjzvyzCZ&uniplatform=NZKPT&language=CHS',
  'title': '中西方媒体报道各国疫情的对比及情感分析方法研究',
  'author': ['陈雪松', '毛佳昕', '马为之', '刘奕群', '张敏', '马少平'],
  'source': '计算机学报',
  'date': '2022-05-15',
  'data': '期刊',
  'quote': '12',
  'download': None,
  'detail': {'摘要': '媒体在信息化社会中扮演了重要角色,应秉承公平、公正、客观的原则.在2020年,媒体的报道引导着大众对新冠疫情的认知和对不同国家防控能力的认识.本文借助搜索引擎收集了26万余条中西方媒体对十个国家疫情情况的报道,分析结果显示,西方媒体在报道中国疫情时,消极文章比例明显偏高;报道美国疫情情况时,新闻特征与其他国家有明显差别.与此同时,较西方媒体,中方媒体在报道不同国家情况时,与实际疫情有更高的一致性,态度更客观.更多还原',
   '关键词': 'COVID-19;情感分析;信息传播;媒体报道;新闻特征;',
   '基金资助': '国家重点研发计划(2018YFC0831700)；国家自然科学基金(61732008,61532011)；北京智源研究院和清华大学国强研究院资助；',
   '专辑': '信息科技',
   '专题': '计算机软件及计算机应用;新闻与传媒',
   '分类号': 'G212;TP391.1'}},
 {'url': 'https://kns.cnki.net/kcms2/article/abstract?v=Nyg97wmOeE707ONVGBvquN7xbyQ7mjOO

In [293]:
import json


with open('subject_count.json','w',encoding='utf-8') as f:
    json.dump(subject_count, f, ensure_ascii=False, indent=4)
with open('articles.json','w',encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)
with open('anisys_data.json','w',encoding='utf-8') as f:
    json.dump(anisys_data, f, ensure_ascii=False, indent=4)
with open('category_count.json','w',encoding='utf-8') as f:
    json.dump(category_count, f, ensure_ascii=False, indent=4)
with open('author_data.json','w',encoding='utf-8') as f:
    json.dump(author_data, f, ensure_ascii=False, indent=4)

In [304]:
article_tab.get('https://kns.cnki.net/kcms2/article/abstract?v=Nyg97wmOeE6QVpIfxMCDnfs8qqMFpQ_Fn_EUSbLXR20MB7yFGha4zDdOFW4M0Ww3IwCvDTnTWmXJnTF8c6Sro4WipK33MGzYcsXKbnSCtSx6C6o9J9ek4liTyO8WvD-vwjW4_BR9NjvNxLuaAGvwwKnz4eIygX-Dgblnb77f2ogkgyqA_98Z_2KKgb1GOmy13vMV6mbtNT3J6nZhNv0PnHY16qaRHN0d&uniplatform=NZKPT')
desc = article_tab.eles('xpath://div[@class="doc"]//div[@class="row"]')
r = []
for i in (desc):
    # print(i.text)
    if '摘要' in i.text:
        r.append('摘要')
        r.append(i.text.split('摘要', 1)[1].strip())
    elif '：' in i.text or '\n' in i.text:
        for j in i.text.split('：'):
            for k in j.split('\n'):
                if k != '':
                    r.append (k.strip())
    else:
        r.append(i.text.strip())
print(r)
r = {r[i]:r[i+1] for i in range(0,len(r),2)}
print(r)
# articles[index]['detail'] = r

['摘要', '：随着互联网的快速发展,网页已成为获取信息的主要来源。为了使出版机构能够及时从大量网页中发现所需文献,需要设计能够从HTML页面中自动提取文献信息的算法。为此,本文设计了基于条件随机场的文献记录分析算法:首先,设计了文档对象树的分割算法,通过分割标记将页面数据分成独立的部分,这些数据块由标签和文本序列构成;随后,将该序列作为条件随机场模型的特征向量,建立文献信息标记模型;最后,设计启发式算法,从标记模型中提取文献信息数据,并通过实验验证了其有效性。更多还原', '关键词', '数字出版;条件随机场;文献记录分析;', '基金', 'r1601000668,北京印刷学院博士启动基金项目,ea201507,北京印刷学院校级重点项目；PXM2016_014223_000025,协同创新中心一绿色印刷与出版技术；', '会议名称', '2016年全国通信软件学术会议', '会议时间', '2016-06-24', '会议地点', '中国陕西西安', '专辑', '信息科技', '专题', '计算机软件及计算机应用', '分类号', 'TP391.1']
{'摘要': '：随着互联网的快速发展,网页已成为获取信息的主要来源。为了使出版机构能够及时从大量网页中发现所需文献,需要设计能够从HTML页面中自动提取文献信息的算法。为此,本文设计了基于条件随机场的文献记录分析算法:首先,设计了文档对象树的分割算法,通过分割标记将页面数据分成独立的部分,这些数据块由标签和文本序列构成;随后,将该序列作为条件随机场模型的特征向量,建立文献信息标记模型;最后,设计启发式算法,从标记模型中提取文献信息数据,并通过实验验证了其有效性。更多还原', '关键词': '数字出版;条件随机场;文献记录分析;', '基金': 'r1601000668,北京印刷学院博士启动基金项目,ea201507,北京印刷学院校级重点项目；PXM2016_014223_000025,协同创新中心一绿色印刷与出版技术；', '会议名称': '2016年全国通信软件学术会议', '会议时间': '2016-06-24', '会议地点': '中国陕西西安', '专辑': '信息科技', '专题': '计算机软件及计算机应用', '分类号': 'TP391.1'}


In [300]:
lll = ['摘要', '为评价一个汉字识别系统的可信度，提出了汉字识别系统误识模型的概念，给出了误识模型的定义。在实验分析的基础上，确立了影响系统识别性能的两个主要参数', '首选距离及其与二选的距离差。给出了运用统计分析方法建立识别系统误识模型的方法。对一个现有的手写体汉字识别系统，经过对其识别性能的统计分析，建立了一个误识模型，初步测试表明了该方法的可行性，达到了实用的要求。误识模型可用于识别率的估计、拒识处理、后处理、多识别器集成和综合评判等多个方面。更多还原', '关键词', '汉字识别；误识模型；后处理;', '基金资助', '国家“八六三”高技术项目,国家自然科学基金；', 'DOI', '10.16511/j.cnki.qhdxxb.1998.s1.026', '专辑', '工程科技Ⅱ辑;信息科技', '专题', '计算机软件及计算机应用', '分类号', 'TP391.41,']
for i in lll:
    print(i)

摘要
为评价一个汉字识别系统的可信度，提出了汉字识别系统误识模型的概念，给出了误识模型的定义。在实验分析的基础上，确立了影响系统识别性能的两个主要参数
首选距离及其与二选的距离差。给出了运用统计分析方法建立识别系统误识模型的方法。对一个现有的手写体汉字识别系统，经过对其识别性能的统计分析，建立了一个误识模型，初步测试表明了该方法的可行性，达到了实用的要求。误识模型可用于识别率的估计、拒识处理、后处理、多识别器集成和综合评判等多个方面。更多还原
关键词
汉字识别；误识模型；后处理;
基金资助
国家“八六三”高技术项目,国家自然科学基金；
DOI
10.16511/j.cnki.qhdxxb.1998.s1.026
专辑
工程科技Ⅱ辑;信息科技
专题
计算机软件及计算机应用
分类号
TP391.41,
